In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.cross_validation import KFold
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_data=pd.read_csv('/Users/s0c02nj/Desktop/WNS/Old_versions/train_LZdllcl.csv')
test_data=pd.read_csv('/Users/s0c02nj/Desktop/WNS/Old_versions/test_2umaH9m.csv')
test_x=test_data
train_x=train_data.drop(columns=['is_promoted'])#-----removing response


In [3]:
combo=pd.concat([train_x,test_x],axis=0,sort=False)
combo.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73


In [4]:
#Replacing Missing with max frequency
combo['previous_year_rating']=combo['previous_year_rating'].fillna(3)
combo['education']=combo['education'].fillna("Bachelor's")

In [5]:
combo['department'] = combo.department.astype('category')
combo['region'] = combo.region.astype('category')
combo['education'] = combo.education.astype('category')
combo['gender'] = combo.gender.astype('category')
combo['recruitment_channel'] = combo.recruitment_channel.astype('category')
#combo['previous_year_rating'] = combo.previous_year_rating.astype('category')
#combo['KPIs_met >80%'] = combo['KPIs_met >80%'].astype('category')
#combo['awards_won?'] = combo['awards_won?'].astype('category')

In [6]:
#Filter
data_categorical = combo.filter(['department','region','education','gender','recruitment_channel'], axis=1)
data_categorical.head()


,department,region,education,gender,recruitment_channel
0,Sales & Marketing,region_7,Master's & above,f,sourcing
1,Operations,region_22,Bachelor's,m,other
2,Sales & Marketing,region_19,Bachelor's,m,sourcing
3,Sales & Marketing,region_23,Bachelor's,m,other
4,Technology,region_26,Bachelor's,m,other


In [7]:
#Label Encode
from sklearn.preprocessing import LabelEncoder
categorical_encoded=data_categorical.apply(LabelEncoder().fit_transform)

In [8]:
#categorical_encoded['previous_year_rating']=combo['previous_year_rating']
categorical_encoded['KPIs_met']=combo['KPIs_met >80%']
categorical_encoded['awards_won']=combo['awards_won?']
categorical_encoded.describe()

,department,region,education,gender,recruitment_channel,KPIs_met,awards_won
count,78298.000000,78298.000000,78298.000000,78298.000000,78298.000000,78298.000000,78298.000000
mean,4.971187,15.439922,0.562428,0.703620,0.867902,0.354032,0.023053
std,2.508570,8.753003,0.890777,0.456664,0.980927,0.478222,0.150073
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,14.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,7.000000,21.000000,2.000000,1.000000,2.000000,1.000000,0.000000
max,8.000000,33.000000,2.000000,1.000000,2.000000,1.000000,1.000000


In [9]:
#Modifying the datatypes
categorical_encoded['department'] = categorical_encoded.department.astype('category')
categorical_encoded['region'] = categorical_encoded.region.astype('category')
categorical_encoded['education'] = categorical_encoded.education.astype('category')
categorical_encoded['gender'] = categorical_encoded.gender.astype('category')
categorical_encoded['recruitment_channel'] = categorical_encoded.recruitment_channel.astype('category')
#categorical_encoded['no_of_trainings']=combo['no_of_trainings']
#categorical_encoded['age']=combo['age']
#categorical_encoded['length_of_service']=combo['length_of_service']
#categorical_encoded['avg_training_score']=combo['avg_training_score']
#categorical_encoded['employee_id']=combo['employee_id']
#cat_encoded.dtypes

In [10]:
categorical_encoded.head()

,department,region,education,gender,recruitment_channel,KPIs_met,awards_won
0,7,31,2,0,2,1,0
1,4,14,0,1,0,0,0
2,7,10,0,1,2,0,0
3,7,15,0,1,0,0,0
4,8,18,0,1,0,0,0


# One hot encoding

In [11]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
one_hot_encoded=enc.fit_transform(categorical_encoded)

In [12]:
one_hot_df=pd.DataFrame(one_hot_encoded.todense())
one_hot_df.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [13]:
#combo['age']
#one_hot_df.head()
#one_hot_df
len(combo['age'])
len(one_hot_df)
#one_hot_df.head()

78298

In [13]:
#one_hot_df.shape
one_hot_df['no_of_trainings']=np.array(combo.no_of_trainings)
one_hot_df['age']=np.array(combo['age'])
one_hot_df['length_of_service']=np.array(combo['length_of_service'])
one_hot_df['avg_training_score']=np.array(combo['avg_training_score'])
one_hot_df['previous_year_rating']=np.array(combo['previous_year_rating'])

In [14]:
one_hot_df.shape

(78298, 60)

In [15]:
#X_train=decom_mat[0:54808]
#X_test=decom_mat[54808:78298]
#X_train=one_hot_df.iloc[0:54808,:]
#X_test=one_hot_df.iloc[54808:78298,:]

In [16]:
#Standardizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(one_hot_df) 

In [ ]:
#Pca
#from sklearn.decomposition import PCA
#from sklearn.decomposition import KernelPCA
#ker_pca = KernelPCA(n_components=10,kernel='rbf')
#decom_mat=ker_pca.fit_transform(scaled_values)

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=47)
decom_mat=pca.fit_transform(scaled_values)

In [21]:
#pca.explained_variance_ratio_.sum()

In [22]:
#from sklearn.decomposition import KernelPCA
#ker_pca = KernelPCA(n_components=10,kernel='rbf')
#decom_mat1=ker_pca.fit_transform(scaled_values)
#from sklearn import manifold
#mds=manifold.MDS(n_components=60)
#decom_mat2=mds.fit_transform(scaled_values)
#from  sklearn.manifold import SpectralEmbedding
#embed=SpectralEmbedding(n_components=60, affinity='nearest_neighbors',n_jobs=6)
#decom_mat2=embed.fit_transform(scaled_values)

In [28]:
X_train=decom_mat[0:54808]
X_test=decom_mat[54808:78298]
Y_train=np.array(train_data.is_promoted)

In [ ]:
print X_train.shape
Y_train=np.array(train_data.is_promoted)


In [29]:
#Model
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
x1_train, x1_val, y1_train, y1_val = train_test_split(X_train, Y_train,test_size = .15,random_state=12)

In [ ]:
#sm = SMOTE(random_state=12, ratio = 0.7)
#res_x, res_y = sm.fit_sample(x1_train, y1_train)

In [24]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier as gbm
gbm_model=gbm(loss='deviance', learning_rate=0.01, n_estimators=600, subsample=1.0, 
              criterion='friedman_mse', min_samples_split=900, min_samples_leaf=10, 
              min_weight_fraction_leaf=0.0, max_depth=15,max_features='sqrt',random_state=12)
gbm_model.fit(x1_train, y1_train)
#X_train

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=15,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=10, min_samples_split=900,
              min_weight_fraction_leaf=0.0, n_estimators=600,
              presort='auto', random_state=12, subsample=1.0, verbose=0,
              warm_start=False)

In [27]:
gbm_model.score(x1_val, y1_val)
print recall_score(y1_val, gbm_model.predict(x1_val))
print precision_score(y1_val, gbm_model.predict(x1_val))
f1_score(y1_val, gbm_model.predict(x1_val))
#cross_val_score(gbm_model,X_train,Y_train,cv=4,scoring='f1')

0.2309899569583931
0.9096045197740112


0.3684210526315789

In [ ]:
tuning.fit(X_train,y_train)
tuning.grid_scores_, tuning.best_params_, tuning.best_score_

In [ ]:
#Model
#from imblearn.over_sampling import SMOTE
#from sklearn.model_selection import train_test_split
#x1_train, x1_val, y1_train, y1_val = train_test_split(res_x, res_y,test_size = .15,random_state=12)

In [ ]:
#x1_train.head()
#x1_train.dtypes

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=10, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False)
log_model.fit(x1_train,y1_train)

In [ ]:
print recall_score(np.array(y1_val), log_model.predict(np.array(x1_val)))
print precision_score(np.array(y1_val), log_model.predict(np.array(x1_val)))
print f1_score(np.array(y1_val), log_model.predict(np.array(x1_val)))

In [24]:
import tensorflow as tf
from keras import backend as K
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.models import Sequential
from keras.layers import Input, Dropout

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [39]:
#Dense layer
np.random.seed(0)
deep_model=Sequential()

In [40]:
from keras.layers import Dense,Activation
deep_model.add(Dense(512,input_shape=(47,),activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(256,activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(128,activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(64,activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(32,activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(16,activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(8,activation='relu'))
deep_model.add(Dropout(0.25))
deep_model.add(Dense(4,activation='relu'))
deep_model.add(Dense(2 ,activation='softmax'))


In [41]:
from keras.optimizers import SGD
#model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
deep_model.summary()
deep_model.compile(loss='categorical_crossentropy',optimizer=SGD(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               24576     
_________________________________________________________________
dropout_21 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_22 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_23 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 64)                8256      
__________

In [42]:
#converting the y to be given as an input
from keras.utils import to_categorical
cat_y=to_categorical(Y_train, num_classes=2)

#x_scaled=sklearn.preprocessing.scale(x, axis=0, with_mean=True, with_std=True, copy=True)
x1_train, x1_test, y1_train, y1_test = train_test_split(X_train, cat_y, test_size=0.2, random_state=42)

In [ ]:
deep_model.fit(x1_train, y1_train,batch_size=30, validation_data=(x1_test,y1_test),epochs=15)

Train on 43846 samples, validate on 10962 samples
Epoch 1/15
43846/43846 [==============================] - 5s 125us/step - loss: 0.3060 - acc: 0.9141 - val_loss: 0.2798 - val_acc: 0.9172
Epoch 2/15
43846/43846 [==============================] - 5s 109us/step - loss: 0.2878 - acc: 0.9142 - val_loss: 0.2657 - val_acc: 0.9172
Epoch 3/15
43846/43846 [==============================] - 5s 107us/step - loss: 0.2782 - acc: 0.9142 - val_loss: 0.2531 - val_acc: 0.9172
Epoch 4/15
43846/43846 [==============================] - 5s 107us/step - loss: 0.2708 - acc: 0.9142 - val_loss: 0.2480 - val_acc: 0.9172
Epoch 5/15
43846/43846 [==============================] - 4s 102us/step - loss: 0.2676 - acc: 0.9142 - val_loss: 0.2452 - val_acc: 0.9172
Epoch 6/15
43846/43846 [==============================] - 4s 101us/step - loss: 0.2653 - acc: 0.9142 - val_loss: 0.2448 - val_acc: 0.9172
Epoch 7/15
43846/43846 [==============================] - 5s 113us/step - loss: 0.2618 - acc: 0.9142 - val_loss: 0.2439 - 

In [ ]:
prediction_deep = deep_model.predict_proba(X_test)[:,1]

In [ ]:
np.savetxt("/Users/s0c02nj/Desktop/WNS/probability_deep.csv",prediction_deep)

In [ ]:
! pip install impyute

In [ ]:
import impyute as impy


In [ ]:
#43846+946

In [ ]:
#Gaussian